In [216]:
import pandas as pd
class DataHandler:
    """
        Get data from GSC Bucket 
    """
    def __init__(self):
        self.csvfile1 = None
        self.csvfile2 = None
        self.gouped_data = None
    def get_data(self):
        print(" - - - fetch data from gcs bucket : - - - ")
        listing = pd.read_csv('datas/listings_final.csv', sep=';')
        prices = pd.read_csv('datas/price_availability.csv', sep=';')
        self.csvfile1, self.csvfile2 = listing, prices  
        return " - - - data loaded - - - \nFiles : \n  - listing {} \n  - prices {}".format(listing.shape,prices.shape)
    def group_data(self):
        data = self.csvfile2.groupby('listing_id')['local_price'].mean()
        self.gouped_data = pd.merge(data, self.csvfile1, on='listing_id')
        print(" - - - data merged - - - ")
    def get_process_data(self):
        self.get_data()
        self.group_data()
        print(" - - - data processed - - - ")

In [217]:
data = DataHandler()
data.get_process_data()
%time 

 - - - fetch data from gcs bucket : - - - 
 - - - data merged - - - 
 - - - data processed - - - 
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


In [218]:
class FeatureRecipe(DataHandler):
    """
    Feature processing class
    """
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.category = None
        self.discrete_variable = None
        self.continous_variable = None
        
    #Useless feature
    def drop_useless(self):
        self.data = self.data.drop(columns=['name', 'Unnamed: 0'])
        print("- - - drop useless columns - - - ")
        
    def deal_duplicate(self):
        self.data = self.data.drop_duplicates(subset ="listing_id") 
        print("- - - drop duplicated index - - - ")
    
    
    #NaN
    def drop_nan(self, thresold: float):
        if (self.data.isna().sum().mean() < thresold):
            self.data = self.data.dropna()
            print("- - - drop NaN values - - - ")
            
    #Separate feature for feature engineering
    def separate_variable_types(self) -> None:
        self.data.select_dtypes(include='bool')*1
        self.category = self.data.select_dtypes(include='object')
        self.discrete_variable = self.data.select_dtypes(include='int64')
        self.continous_variable = self.data.select_dtypes(include='float64')
        print("- - - separate variable types - - - ")

    #Datetime
    def deal_dtime(self):
        pass
    
    def prepare_data(self, thresold: float):
        """
        Wrap code above
        """
        self.drop_useless()
        self.deal_duplicate()
        self.drop_nan(thresold)
        self.separate_variable_types()
        self.deal_dtime()
        print("- - - data processed - - -")

In [219]:
recipe = FeatureRecipe(data.gouped_data)
recipe.prepare_data(6)
%time 

- - - drop useless columns - - - 
- - - drop duplicated index - - - 
- - - drop NaN values - - - 
- - - separate variable types - - - 
- - - data processed - - -
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs
